- Assume the path of trackcluster is path/to/trackcluster
- Change the path/to/github to real $PATH of trackcluster location in your own run


#### files needed
1. The reference file: reference.fa
2. the nanopore read file: nanopore_reads.fa
3. reference annotation file: anno.gff
4. Optional:the faSize output of reads from each groups; the matching score of the full-length indicator splicing leader 

In [ ]:
# add the trackcluster to the python env
import sys
sys.path.insert(0, "path/to/") 
import trackcluster

In [ ]:
%%bash
# run the minimap2 with 10 cores
minimap2 -ax splice -k14 -uf -t 10 \
    reference.fa \
    nanopore_reads.fa > aln.sam
samtools view -bS aln.sam>aln.bam
samtools sort aln.bam >aln_s.bam
samtools index aln_s.bam

In [ ]:
%%bash
# convert the bam file to the track file in bigGenPred format, the bed file can be visualized in IGV or UCSC
python path/to/trackcluster/script/bam2bigg.py -b aln_s.bam -o nano.bed

In [ ]:
%%bash
#convert the reference gff file to the track file in bigGenPred format, the bed file can be visualized in IGV or UCSC
python path/to/trackcluster/script/gff2bigg.py -k Gene -i anno.gff -o anno.bed 

In [ ]:
%%bash
# run a quick bedtool check to assign gene name to each reads
bedtools sort -i nano.bed > nanos.bed
bedtools sort -i anno.bed > annos.bed
bedtools intersect -wa -wb -r  -f 0.05 -a nanos.bed -b annos.bed > nano_inter_ref.bed

In [ ]:
# get the dic of read:gene
read_gene={}
f=open("nano_inter_ref.bed")
for line in f.readlines():
    line_l=line.split("\t")
    name=line_l[3]
    gene=line_l[-3]
    read_gene[name] = gene
f.close()

In [ ]:
# Optional, get the dic of read:group, if any
read_n={}

f=open("group1.fa.sizes")
for line in f.readlines():
    line_l=line.split("\t")
    name=line_l[0]
    group="group1"
    read_n[name] = group
f.close()

f=open("group2.fa.sizes")
for line in f.readlines():
    line_l=line.split("\t")
    name=line_l[0]
    group="group12"
    read_n[name] = group
f.close()

In [ ]:
# Optional, get the dic of read:score, if any
read_score={}
f=open("SL_score.txt")
for line in f.readlines():
    line_l=line.split("\t")
    name=line_l[0]
    score=int(line_l[1])
    read_score[name] = score
f.close()

In [ ]:
from trackcluster.tracklist import read_bigg, write_bigg, bigglist_to_bedfile

In [ ]:
nano_bigg=read_bigg("nanos.bed")

In [ ]:
# add the infor from read_gene, read_n to the track file
nano_bigg_new=[]
for bigg in nano_bigg:
    try:
        gene=read_gene[bigg.name]
        group=read_n[bigg.name] # Optional
        score=read_score[bigg.name] # Optional

        bigg.geneName=gene
        bigg.geneName2=group # Optional
        bigg.score=score # Optional
        
        nano_bigg_new.append(bigg)
    except KeyError:
        pass 

In [ ]:
# write the new file with group information
write_bigg(nano_bigg_new, "nanogroup.bed")

In [ ]:
from collections import OrderedDict
def group_bigg_by_gene(bigglist):
    gene_bigg=OrderedDict()
    
    for bigg in bigglist:
        try:
            gene_bigg[bigg.geneName].append(bigg)
        except KeyError:
            gene_bigg[bigg.geneName]=[]
            gene_bigg[bigg.geneName].append(bigg)
    return gene_bigg

In [ ]:
gene_nano=group_bigg_by_gene(nano_bigg_new)

In [ ]:
anno_bigg=read_bigg("annos.bed")

In [ ]:
gene_anno=group_bigg_by_gene(anno_bigg)

In [ ]:
# use the gene_anno and gene_nano to write the small trackgroups for each genes
for gene, nano_bigg in gene_nano.iteritems():
    anno_bigg=gene_anno[gene]
    try:
        os.mkdir(gene)
    except OSError:
        pass
    
    anno_out="./{gene}/{gene}_gff.bed".format(gene=gene)
    nano_out="./{gene}/{gene}_nano.bed".format(gene=gene)
    
    write_bigg(anno_bigg, anno_out)
    write_bigg(nano_bigg, nano_out)

In [ ]:
from trackcluster.batch import *

In [ ]:
subk=gene_nano.keys()
len(subk)

In [ ]:
# run the clustering for each gene 
errors_ll=[]

def process_one_subsample_try(key, intronweight=0.5, by="ratio_all", full=False):
    print key
    gff_file = "./" + key + "/" + key + "_gff.bed"
    nano_file = "./" + key + "/" + key + "_nano.bed"
    figout = "./" + key + "/" + key + "_coverage.pdf"
    biggout = "./" + key + "/" + key + "_simple_coverage2.bed"
    Dout = "./" + key + "/" + key + "_simple_coverage2.csv"

    if full is False:
        if os.stat(nano_file).st_size == 0:  # no bigg nano file
            return 0
        if os.path.isfile(biggout):  # already processed
            return 0

    bigg_gff = read_bigg(gff_file)
    bigg_nano_raw = read_bigg(nano_file)

    try:
        bigg_nano = prefilter_smallexon(bigg_nano_raw, bigg_gff, cutoff=50)
        n_count = 100
        n = 0

        if bigg_nano is None:
            return 0

        try:
            while n < n_count and len(bigg_nano) > batchsize:
                # print "n=", n
                bigg_1 = bigg_nano[:batchsize]
                bigg_2 = bigg_nano[batchsize:]
                _, bigg_list_by1 = flow_cluster(bigg_1, bigg_gff, by, intronweight=intronweight)
                bigg_nano = add_subread_bigg(bigg_list_by1 + bigg_2)
                n += 1
                
            D, bigg_nano_new = flow_cluster(bigg_nano, bigg_gff, by, intronweight=intronweight)
            bigg_nano_new = add_subread_bigg(bigg_nano_new)

            ### save nessary files
            for bigg in bigg_nano_new:
                bigg.write_subread()

            bigg_count_write(bigg_nano_new, out=biggout)
        except Exception as e:
            errors_ll.append((key,e))
    except Exception as e:
        errors_ll.append((key,e))

    return 1

In [ ]:
from trackcluster.utils import parmap
# run the clustring using 40 cores
parmap(process_one_subsample_try, subk, 40)